In [ ]:
## AIIB InfraTech Portal – Data Collection Pipeline

This notebook builds a Python data pipeline to:
- Ingest company-level data from the AIIB InfraTech public API
- Handle pagination and request throttling
- Normalise nested JSON into tabular form
- Apply text sanitisation for Excel compatibility
- Export clean datasets for analysis

Note: The company list endpoint does not expose thematic tags; 
full classification would require enrichment via a detail-level API.

In [ ]:
import requests
import pandas as pd

In [ ]:
import sys
print(sys.executable)


In [ ]:
!pip3 install requests pandas openpyxl


In [ ]:
import requests
import pandas as pd


In [ ]:
import requests
import pandas as pd
import time

URL = "https://www.infratechportal.org/itp/providerInfo/companyList"

# Base payload from your DevTools
BASE_PAYLOAD = {
    "name": "",
    "orderField": "timeDesc",
    "pageNo": 1,
    "pageSize": 10,       # will override to 100 per page
    "status": "",
    "type": "",
    "subSectorList": [],
    "subThemeList": [],
    "tagList": [],
    "techTags": [],
    "isSearch": False,
    "keyWord": "",
}


In [ ]:
import requests
import pandas as pd
import time

URL = "https://www.infratechportal.org/itp/providerInfo/companyList"

# Base payload from your DevTools
BASE_PAYLOAD = {
    "name": "",
    "orderField": "timeDesc",
    "pageNo": 1,
    "pageSize": 10,       # will override to 100 per page
    "status": "",
    "type": "",
    "subSectorList": [],
    "subThemeList": [],
    "tagList": [],
    "techTags": [],
    "isSearch": False,
    "keyWord": "",
    "queryType": "1"
}

HEADERS = {
    "User-Agent": "Mozilla/5.0",
    "Content-Type": "application/json"
}

all_data = []
page = 1
page_size = 100  # Faster: 100 per page


while True:
    print(f"Fetching page {page}...")

    payload = BASE_PAYLOAD.copy()
    payload["pageNo"] = page
    payload["pageSize"] = page_size

    r = requests.post(URL, json=payload, headers=HEADERS)
    r.raise_for_status()

    j = r.json()
    items = j.get("data", [])

    if not items:
        print("No more data returned.")
        break

    all_data.extend(items)

    total = j.get("total", None)
    if total and len(all_data) >= total:
        print("Reached total count.")
        break

    page += 1
    time.sleep(0.2)   # avoid hitting server too fast


# Convert to DataFrame
df = pd.json_normalize(all_data)
print(f"Downloaded {len(df)} companies.")

import re

ILLEGAL = re.compile(r"[\x00-\x08\x0B\x0C\x0E-\x1F]")

def clean_excel_text(x):
    if isinstance(x, str):
        return ILLEGAL.sub(" ", x).replace("\u00a0", " ").strip()
    return x

# Clean for Excel
df_clean = df.map(clean_excel_text)

# Save cleaned data
df_clean.to_excel("infratech_companies.xlsx", index=False)
df_clean.to_csv("infratech_companies.csv", index=False)

df_clean.head()


In [ ]:
import re

# Remove all illegal Excel control characters
illegal_chars = re.compile(r'[\x00-\x08\x0B\x0C\x0E-\x1F]')

for col in df.columns:
    df[col] = df[col].astype(str).apply(lambda x: illegal_chars.sub("", x))


In [ ]:
df.to_excel("infratech_companies.xlsx", index=False)
df.to_csv("infratech_companies.csv", index=False)


In [ ]:
import requests
import pandas as pd
import time
import re

URL = "https://www.infratechportal.org/itp/providerInfo/companyList"

BASE_PAYLOAD = {
    "name": "",
    "orderField": "timeDesc",
    "pageNo": 1,
    "pageSize": 10,
    "status": "",
    "type": "",
    "subSectorList": [],
    "subThemeList": [],
    "tagList": [],
    "techTags": [],
    "isSearch": False,
    "keyWord": "",
    "queryType": "1"
}

HEADERS = {
    "User-Agent": "Mozilla/5.0",
    "Content-Type": "application/json"
}

all_data = []
page = 1
page_size = 100

while True:
    print(f"Fetching page {page}...")

    payload = BASE_PAYLOAD.copy()
    payload["pageNo"] = page
    payload["pageSize"] = page_size

    r = requests.post(URL, json=payload, headers=HEADERS)
    r.raise_for_status()

    j = r.json()
    items = j.get("data", [])

    if not items:
        print("No more data returned.")
        break

    all_data.extend(items)

    total = j.get("total", None)
    if total and len(all_data) >= total:
        print("Reached total count.")
        break

    page += 1
    time.sleep(0.2)

df = pd.json_normalize(all_data)
print(f"Downloaded {len(df)} companies.")

# CLEAN ILLEGAL CHARACTERS
illegal_chars = re.compile(r'[\x00-\x08\x0B\x0C\x0E-\x1F]')
for col in df.columns:
    df[col] = df[col].astype(str).apply(lambda x: illegal_chars.sub("", x))

# SAVE
df.to_excel("infratech_companies.xlsx", index=False)
df.to_csv("infratech_companies.csv", index=False)

df.head()


In [ ]:
## AIIB InfraTech Portal – Data Collection Pipeline

This notebook builds a Python data pipeline to:
- Ingest company-level data from the AIIB InfraTech public API
- Handle pagination and request throttling
- Normalise nested JSON into tabular form
- Apply text sanitisation for Excel compatibility
- Export clean datasets for analysis

Note: The company list endpoint does not expose thematic tags; 
full classification would require enrichment via a detail-level API.


In [ ]:
len(df)

In [ ]:
df.columns


In [ ]:
df.columns.tolist()
